## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-11-18-56-32 +0000,nypost,Elderly blind woman regains sight thanks to ra...,https://nypost.com/2025/08/11/world-news/blind...
1,2025-08-11-18-55-43 +0000,nyt,At Least One Dead in U.S. Steel Plant Explosio...,https://www.nytimes.com/2025/08/11/us/us-steel...
2,2025-08-11-18-54-00 +0000,wsj,Israel Killed Five Al Jazeera Journalists in A...,https://www.wsj.com/world/middle-east/israel-k...
3,2025-08-11-18-52-00 +0000,wsj,"At Least One Killed, Dozens Injured in Explosi...",https://www.wsj.com/business/pennsylvania-us-s...
4,2025-08-11-18-52-00 +0000,wsj,Stock Market Today: Trump Extends China Deadli...,https://www.wsj.com/livecoverage/stock-market-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
33,trump,45
94,he,14
35,china,13
40,new,11
223,out,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
20,2025-08-11-18-22-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...,110
198,2025-08-11-01-00-00 +0000,wsj,News the jobs market was much weaker than prev...,https://www.wsj.com/economy/jobs-inflation-tru...,99
69,2025-08-11-16-09-47 +0000,nypost,Trump says he’ll tell Putin to end Ukraine inv...,https://nypost.com/2025/08/11/world-news/trump...,96
12,2025-08-11-18-35-01 +0000,startribune,Trump says he's placing Washington police unde...,https://www.startribune.com/trump-is-promising...,96
217,2025-08-10-20-57-00 +0000,wsj,Intel CEO Lip-Bu Tan is set to visit the White...,https://www.wsj.com/tech/intel-ceo-trump-white...,96


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
20,110,2025-08-11-18-22-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...
63,50,2025-08-11-16-34-28 +0000,nypost,"At least one dead, dozens trapped or injured a...",https://nypost.com/2025/08/11/us-news/people-t...
217,46,2025-08-10-20-57-00 +0000,wsj,Intel CEO Lip-Bu Tan is set to visit the White...,https://www.wsj.com/tech/intel-ceo-trump-white...
228,44,2025-08-10-20-08-00 +0000,wsj,Israel’s New War Plan Includes Fighting in Gaz...,https://www.wsj.com/world/middle-east/israels-...
69,37,2025-08-11-16-09-47 +0000,nypost,Trump says he’ll tell Putin to end Ukraine inv...,https://nypost.com/2025/08/11/world-news/trump...
37,36,2025-08-11-17-47-23 +0000,bbc,UN condemns targeted Israeli attack that kille...,https://www.bbc.com/news/articles/cq688qz3rlro...
111,36,2025-08-11-13-46-00 +0000,wsj,Colombian Presidential Hopeful Dies Two Months...,https://www.wsj.com/world/americas/colombian-p...
25,34,2025-08-11-18-15-40 +0000,nyt,Tariffs on China Set to Rise Sharply Tuesday i...,https://www.nytimes.com/2025/08/11/us/politics...
169,32,2025-08-11-08-52-43 +0000,wapo,Here are countries that recognize a Palestinia...,https://www.washingtonpost.com/world/2025/07/3...
89,28,2025-08-11-15-02-46 +0000,nypost,Wild video shows Chinese ships crashing while ...,https://nypost.com/2025/08/11/world-news/chine...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
